In [1]:
import os
import util
# import model_builder
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Input, TimeDistributed, GRU, Dense, Dropout, Lambda, GlobalAveragePooling2D, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score

In [2]:
base_dir = 'other_artifacts'
data = os.path.join(base_dir, 'split_3d_data.pkl')
batch_size = 32

In [3]:
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Load Data

In [4]:
X_train, X_val, X_test, y_train, y_val, y_test = util.load_split_3d_data(data)

In [5]:
X_train.shape, X_test.shape, X_val.shape

((280, 16, 224, 224, 3), (60, 16, 224, 224, 3), (60, 16, 224, 224, 3))

In [6]:
num_frames, img_size = X_train.shape[1], X_train.shape[2:4]
print(num_frames, img_size)

16 (224, 224)


In [7]:
y_train.shape, y_test.shape, y_val.shape

((280,), (60,), (60,))

In [8]:
num_train, num_val, num_test = X_train.shape[0], X_val.shape[0], X_test.shape[0]
num_train, num_val, num_test

(280, 60, 60)

## Factorizing Target

In [9]:
labels = pd.factorize(y_val)[1]
print(labels)

['real' 'fake']


In [10]:
y_train, y_val, y_test = pd.factorize(y_train)[0], pd.factorize(y_val)[0], pd.factorize(y_test)[0]

In [11]:
y_train.shape, y_test.shape, y_val.shape

((280,), (60,), (60,))

# MobileNetV2 & GRU

In [12]:
base_cnn = keras.applications.MobileNetV2(
    input_shape=img_size+(3,),
    include_top=False,
    weights='imagenet')
# unfreezing a few of the last layers
base_cnn.trainable = False
for layer in base_cnn.layers[:-30]:
    layer.trainable = True

In [13]:
# adding data augmentation for make model more robust
data_augmentation = Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
    keras.layers.RandomBrightness(0.2),
    keras.layers.RandomContrast(0.2)
])

In [14]:
(num_frames,)+img_size+(3,)

(16, 224, 224, 3)

In [15]:
model = Sequential()
model.add(Input(shape=(num_frames,)+img_size+(3,)))
# Using TimeDistributed to apply Augmentation, MobileNetV2 Preprocessing, & MobileNetV2 CNN frame-by-frame
model.add(TimeDistributed(data_augmentation))
model.add(TimeDistributed(Lambda(keras.applications.mobilenet_v2.preprocess_input)))
model.add(TimeDistributed(base_cnn))
model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(GRU(256, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, 16, 224, 224, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 16, 224, 224, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 16, 7, 7, 1280)      │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, 16, 1280)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 256)                 │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,472,193 (13.25 MB)

 Trainable params: 1,911,681 (7.29 MB)

 Non-trainable params: 1,560,512 (5.95 MB)

In [16]:
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [ ]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=5,
                      restore_best_weights=True, verbose=1)
model.fit(X_train, y_train, 
           validation_data=(X_val, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", test_accuracy)
print("Test Loss:", test_loss)

In [20]:
model.save('artifacts/mobilenetv2_gru.keras')

**Didn't perform as good as the majority voting technique. The reason is the small size of the dataset (just 400 videos in total). Deep Neural Networks particularly one that includes some type of RNNs will require several hundred thousands of samples to train and generalize well.**

* We know that the MobileNetV2 model is able to extract features from the frames, because it is already trained on ImageNet (1.2M images), hence, it knows how to see.
* So, instead of training that base model again and again and wasting the 400 videos teaching it basic vision, we can simply obtain those features as 'embeddings'.Then these embeddings can be passed to the GRU or any other model for the final classification task.
* The use of this is that we can add regularization and try tuning without having to include the training of base-CNN as well, reducing the training time and number of learnable parameters.
* Each embedding vector will be like a semantic summary of the video frames. So, feeding those instead of raw pixels, should make it easier for the classifier to learn, even with little data.

# Embeddings & GRU

In [15]:
def get_embeddings(X_frames, batch_size, num_data, num_frames):
    base_cnn = keras.applications.MobileNetV2(
        input_shape=img_size+(3,),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    preprocessed = keras.applications.mobilenet_v2.preprocess_input(X_frames)
    embeddings = base_cnn.predict(preprocessed, batch_size=batch_size, verbose=1)
    print("Embeddings shape:",embeddings.shape)
    return embeddings.reshape(num_data, num_frames, -1)

In [17]:
train_frames, val_frames, test_frames = util.convert_3d_to_2d(split=3,
                                                         train=(X_train, y_train),
                                                         val=(X_val,y_val),
                                                         test=(X_test, y_test))
X_train_frames, _ = train_frames
X_val_frames, _ = val_frames
X_test_frames, _ = test_frames
X_train_frames.shape, X_val_frames.shape, X_test_frames.shape

((4480, 224, 224, 3), (960, 224, 224, 3), (960, 224, 224, 3))

In [19]:
num_train

280

In [18]:
train_embeddings = get_embeddings(X_train_frames, batch_size, num_train, num_frames)
train_embeddings.shape

140/140 ━━━━━━━━━━━━━━━━━━━━ 33s 215ms/step
Embeddings shape: (4480, 1280)


(280, 16, 1280)

In [15]:
val_embeddings = get_embeddings(X_val_frames, 8, num_val, num_frames)
val_embeddings.shape

120/120 ━━━━━━━━━━━━━━━━━━━━ 20s 136ms/step
Embeddings shape: (960, 1280)


(60, 16, 1280)

In [16]:
test_embeddings = get_embeddings(X_val_frames, 8, num_val, num_frames)
test_embeddings.shape

120/120 ━━━━━━━━━━━━━━━━━━━━ 26s 194ms/step
Embeddings shape: (960, 1280)


(60, 16, 1280)

In [17]:
num_features = train_embeddings.shape[-1]
num_features

1280

In [95]:
LR = 0.00001

In [96]:
model1 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(128, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_49 (GRU)                         │ (None, 128)                 │         541,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_55 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_60 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 541,569 (2.07 MB)

 Trainable params: 541,569 (2.07 MB)

 Non-trainable params: 0 (0.00 B)

In [97]:
model1.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model1.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 29s 403ms/step - accuracy: 0.4821 - loss: 0.8666 - val_accuracy: 0.4667 - val_loss: 0.7930
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.4714 - loss: 0.8088 - val_accuracy: 0.4500 - val_loss: 0.7853
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 187ms/step - accuracy: 0.4964 - loss: 0.7754 - val_accuracy: 0.4167 - val_loss: 0.7822
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.5000 - loss: 0.7860 - val_accuracy: 0.4167 - val_loss: 0.7802
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.4893 - loss: 0.7676 - val_accuracy: 0.4333 - val_loss: 0.7781
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - accuracy: 0.5393 - loss: 0.7696 - val_accuracy: 0.4333 - val_loss: 0.7761
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step - accuracy: 0.4857 - loss: 0.7740 - val_accuracy: 0.4333 - val_loss: 0.7746
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.5321 - loss: 0.7375 - val_accuracy: 0.4500 - v

In [98]:
test_loss1, test_accuracy1 = model1.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy1)
print("Test Loss:", test_loss1)

Test Accuracy: 0.550000011920929
Test Loss: 0.718789279460907


In [99]:
model2 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model2.summary()

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_50 (GRU)                         │ (None, 256)                 │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_56 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_61 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_57 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_62 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,214,209 (4.63 MB)

 Trainable params: 1,214,209 (4.63 MB)

 Non-trainable params: 0 (0.00 B)

In [100]:
model2.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model2.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 24s 634ms/step - accuracy: 0.4857 - loss: 0.9411 - val_accuracy: 0.5167 - val_loss: 0.7769
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 172ms/step - accuracy: 0.5000 - loss: 0.8654 - val_accuracy: 0.5333 - val_loss: 0.7458
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 202ms/step - accuracy: 0.4750 - loss: 0.8590 - val_accuracy: 0.5833 - val_loss: 0.7280
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 155ms/step - accuracy: 0.5179 - loss: 0.8107 - val_accuracy: 0.5500 - val_loss: 0.7172
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.5179 - loss: 0.7987 - val_accuracy: 0.5167 - val_loss: 0.7107
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - accuracy: 0.5214 - loss: 0.7749 - val_accuracy: 0.5000 - val_loss: 0.7083
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.5357 - loss: 0.7852 - val_accuracy: 0.4667 - val_loss: 0.7080
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.4893 - loss: 0.8599 - val_accuracy: 0.4333 - 

In [101]:
test_loss2, test_accuracy2 = model2.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy2)
print("Test Loss:", test_loss2)

Test Accuracy: 0.6166666746139526
Test Loss: 0.67915278673172


In [102]:
model3 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model3.summary()

Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_51 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_52 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_58 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_63 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,576,193 (6.01 MB)

 Trainable params: 1,576,193 (6.01 MB)

 Non-trainable params: 0 (0.00 B)

In [103]:
model3.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model3.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 32s 895ms/step - accuracy: 0.5643 - loss: 0.7153 - val_accuracy: 0.4500 - val_loss: 0.7214
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 231ms/step - accuracy: 0.5036 - loss: 0.7490 - val_accuracy: 0.4667 - val_loss: 0.7136
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 204ms/step - accuracy: 0.5179 - loss: 0.7347 - val_accuracy: 0.4667 - val_loss: 0.7093
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 225ms/step - accuracy: 0.4964 - loss: 0.7296 - val_accuracy: 0.5000 - val_loss: 0.7062
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step - accuracy: 0.5036 - loss: 0.7366 - val_accuracy: 0.5167 - val_loss: 0.7035
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 177ms/step - accuracy: 0.5321 - loss: 0.7219 - val_accuracy: 0.5167 - val_loss: 0.7008
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 188ms/step - accuracy: 0.5536 - loss: 0.7049 - val_accuracy: 0.5167 - val_loss: 0.6982
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.4857 - loss: 0.7184 - val_accuracy: 0.5000 - v

In [104]:
test_loss3, test_accuracy3 = model3.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy3)
print("Test Loss:", test_loss3)

Test Accuracy: 0.6666666865348816
Test Loss: 0.6623032689094543


In [105]:
model4 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model4.summary()

Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_53 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_54 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_59 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_64 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_60 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_65 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,608,961 (6.14 MB)

 Trainable params: 1,608,961 (6.14 MB)

 Non-trainable params: 0 (0.00 B)

In [106]:
model4.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model4.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 41s 527ms/step - accuracy: 0.4643 - loss: 0.7880 - val_accuracy: 0.4667 - val_loss: 0.7058
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.5357 - loss: 0.7671 - val_accuracy: 0.5000 - val_loss: 0.7021
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.4714 - loss: 0.7720 - val_accuracy: 0.4833 - val_loss: 0.7005
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.4821 - loss: 0.7670 - val_accuracy: 0.5167 - val_loss: 0.7004
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.4786 - loss: 0.7529 - val_accuracy: 0.4833 - val_loss: 0.7003
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.4893 - loss: 0.7445 - val_accuracy: 0.4500 - val_loss: 0.6998
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step - accuracy: 0.4893 - loss: 0.7343 - val_accuracy: 0.4667 - val_loss: 0.6995
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.4857 - loss: 0.7409 - val_accuracy: 0.4667 - val_

In [107]:
test_loss4, test_accuracy4 = model4.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy4)
print("Test Loss:", test_loss4)

Test Accuracy: 0.5666666626930237
Test Loss: 0.6789262294769287


In [108]:
model5 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model5.summary()

Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_55 (GRU)                         │ (None, 256)                 │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_61 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_66 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_62 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_67 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_63 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_68 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,001 (4.88 MB)

 Trainable params: 1,280,001 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [109]:
model5.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model5.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 28s 486ms/step - accuracy: 0.4821 - loss: 0.8488 - val_accuracy: 0.4500 - val_loss: 0.7103
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.4250 - loss: 0.9178 - val_accuracy: 0.4167 - val_loss: 0.7098
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.5250 - loss: 0.7944 - val_accuracy: 0.4500 - val_loss: 0.7090
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 190ms/step - accuracy: 0.5071 - loss: 0.8340 - val_accuracy: 0.4500 - val_loss: 0.7085
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - accuracy: 0.5321 - loss: 0.7924 - val_accuracy: 0.4833 - val_loss: 0.7083
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.5429 - loss: 0.7688 - val_accuracy: 0.5333 - val_loss: 0.7085
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5036 - loss: 0.8493 - val_accuracy: 0.5667 - val_loss: 0.7087
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.5179 - loss: 0.8050 - val_accuracy: 0.6000 - val_

In [110]:
test_loss5, test_accuracy5 = model5.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy5)
print("Test Loss:", test_loss5)

Test Accuracy: 0.4833333194255829
Test Loss: 0.7083225250244141


In [137]:
model6 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model6.summary()

Model: "sequential_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_76 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_77 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_85 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_93 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_86 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_94 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,609,985 (6.14 MB)

 Trainable params: 1,609,473 (6.14 MB)

 Non-trainable params: 512 (2.00 KB)

In [138]:
model6.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model6.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 47s 862ms/step - accuracy: 0.5250 - loss: 0.8840 - val_accuracy: 0.5667 - val_loss: 0.6921
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.5000 - loss: 0.8670 - val_accuracy: 0.5833 - val_loss: 0.6921
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 208ms/step - accuracy: 0.4571 - loss: 0.8617 - val_accuracy: 0.5667 - val_loss: 0.6918
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 182ms/step - accuracy: 0.5286 - loss: 0.8365 - val_accuracy: 0.5667 - val_loss: 0.6911
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 170ms/step - accuracy: 0.5143 - loss: 0.8446 - val_accuracy: 0.5667 - val_loss: 0.6902
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 177ms/step - accuracy: 0.5429 - loss: 0.8208 - val_accuracy: 0.5667 - val_loss: 0.6895
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 191ms/step - accuracy: 0.5464 - loss: 0.7691 - val_accuracy: 0.5667 - val_loss: 0.6894
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 165ms/step - accuracy: 0.5500 - loss: 0.7803 - val_accuracy: 0.5500 - 

In [139]:
test_loss6, test_accuracy6 = model6.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy6)
print("Test Loss:", test_loss6)

Test Accuracy: 0.5833333134651184
Test Loss: 0.6865931749343872


In [114]:
model7 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model7.summary()

Model: "sequential_30"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_58 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_59 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_64 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_71 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_65 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_72 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_66 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_73 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,674,753 (6.39 MB)

 Trainable params: 1,674,753 (6.39 MB)

 Non-trainable params: 0 (0.00 B)

In [115]:
model7.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model7.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 32s 670ms/step - accuracy: 0.4929 - loss: 0.7564 - val_accuracy: 0.4667 - val_loss: 0.7015
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - accuracy: 0.5250 - loss: 0.7444 - val_accuracy: 0.4500 - val_loss: 0.7020
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 187ms/step - accuracy: 0.5286 - loss: 0.7112 - val_accuracy: 0.4500 - val_loss: 0.7020
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 214ms/step - accuracy: 0.4750 - loss: 0.7571 - val_accuracy: 0.4500 - val_loss: 0.7013
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 206ms/step - accuracy: 0.4821 - loss: 0.7547 - val_accuracy: 0.4500 - val_loss: 0.7010
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 190ms/step - accuracy: 0.4857 - loss: 0.7636 - val_accuracy: 0.4333 - val_loss: 0.7008
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 183ms/step - accuracy: 0.4929 - loss: 0.7485 - val_accuracy: 0.4333 - val_loss: 0.7004
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 175ms/step - accuracy: 0.5321 - loss: 0.7376 - val_accuracy: 0.4333 - 

In [116]:
test_loss7, test_accuracy7 = model7.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy7)
print("Test Loss:", test_loss7)

Test Accuracy: 0.46666666865348816
Test Loss: 0.6986681818962097


In [117]:
model8 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model8.summary()

Model: "sequential_31"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_60 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_61 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_67 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_74 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_68 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_75 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_69 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_76 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_70 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_77 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,871,873 (7.14 MB)

 Trainable params: 1,871,873 (7.14 MB)

 Non-trainable params: 0 (0.00 B)

In [118]:
model8.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model8.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 46s 879ms/step - accuracy: 0.5179 - loss: 0.7840 - val_accuracy: 0.5000 - val_loss: 0.7045
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step - accuracy: 0.5036 - loss: 0.7731 - val_accuracy: 0.5000 - val_loss: 0.7025
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.4964 - loss: 0.7820 - val_accuracy: 0.5167 - val_loss: 0.7012
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 196ms/step - accuracy: 0.4893 - loss: 0.7772 - val_accuracy: 0.4833 - val_loss: 0.7003
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 189ms/step - accuracy: 0.4786 - loss: 0.7339 - val_accuracy: 0.4833 - val_loss: 0.6995
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 179ms/step - accuracy: 0.5321 - loss: 0.7303 - val_accuracy: 0.5167 - val_loss: 0.6987
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 172ms/step - accuracy: 0.5107 - loss: 0.7658 - val_accuracy: 0.5167 - val_loss: 0.6982
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 184ms/step - accuracy: 0.4929 - loss: 0.7352 - val_accuracy: 0.4500 - 

In [119]:
test_loss8, test_accuracy8 = model8.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy8)
print("Test Loss:", test_loss8)

Test Accuracy: 0.5166666507720947
Test Loss: 0.6958596706390381


In [120]:
model9 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model9.summary()

Model: "sequential_32"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_62 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_63 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_64 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_71 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_78 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_72 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_79 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,003,713 (7.64 MB)

 Trainable params: 2,003,713 (7.64 MB)

 Non-trainable params: 0 (0.00 B)

In [121]:
model9.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model9.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 50s 920ms/step - accuracy: 0.5179 - loss: 0.7134 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 228ms/step - accuracy: 0.4821 - loss: 0.7267 - val_accuracy: 0.5333 - val_loss: 0.6927
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 154ms/step - accuracy: 0.5393 - loss: 0.6992 - val_accuracy: 0.5500 - val_loss: 0.6922
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 187ms/step - accuracy: 0.5214 - loss: 0.7086 - val_accuracy: 0.5167 - val_loss: 0.6914
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - accuracy: 0.5071 - loss: 0.7084 - val_accuracy: 0.5333 - val_loss: 0.6909
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 183ms/step - accuracy: 0.5429 - loss: 0.7007 - val_accuracy: 0.5333 - val_loss: 0.6907
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 281ms/step - accuracy: 0.4929 - loss: 0.7160 - val_accuracy: 0.5333 - val_loss: 0.6905
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 285ms/step - accuracy: 0.5107 - loss: 0.7017 - val_accuracy: 0.5500 - 

In [122]:
test_loss9, test_accuracy9 = model9.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy9)
print("Test Loss:", test_loss9)

Test Accuracy: 0.6000000238418579
Test Loss: 0.6747491955757141


In [123]:
model10 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model10.summary()

Model: "sequential_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_65 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_66 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_67 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_73 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_80 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_74 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_81 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_75 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_82 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,069,505 (7.89 MB)

 Trainable params: 2,069,505 (7.89 MB)

 Non-trainable params: 0 (0.00 B)

In [124]:
model10.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model10.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.4357 - loss: 0.7634 - val_accuracy: 0.5333 - val_loss: 0.6996
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 243ms/step - accuracy: 0.5250 - loss: 0.7158 - val_accuracy: 0.5333 - val_loss: 0.6972
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 233ms/step - accuracy: 0.5357 - loss: 0.7139 - val_accuracy: 0.5333 - val_loss: 0.6953
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.4893 - loss: 0.7346 - val_accuracy: 0.5167 - val_loss: 0.6933
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.4929 - loss: 0.7228 - val_accuracy: 0.5167 - val_loss: 0.6916
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 218ms/step - accuracy: 0.4893 - loss: 0.7166 - val_accuracy: 0.5167 - val_loss: 0.6905
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 230ms/step - accuracy: 0.5571 - loss: 0.6982 - val_accuracy: 0.5167 - val_loss: 0.6898
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 230ms/step - accuracy: 0.4500 - loss: 0.7459 - val_accuracy: 0.5167 - val

In [125]:
test_loss10, test_accuracy10 = model10.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy10)
print("Test Loss:", test_loss10)

Test Accuracy: 0.6000000238418579
Test Loss: 0.6774458289146423


In [126]:
model11 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model11.summary()

Model: "sequential_34"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_68 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_69 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_70 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_76 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_83 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_77 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_84 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_78 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_85 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_79 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_86 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,266,625 (8.65 MB)

 Trainable params: 2,266,625 (8.65 MB)

 Non-trainable params: 0 (0.00 B)

In [127]:
model11.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model11.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.5036 - loss: 0.7306 - val_accuracy: 0.5000 - val_loss: 0.6937
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 263ms/step - accuracy: 0.4964 - loss: 0.7305 - val_accuracy: 0.5167 - val_loss: 0.6939
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 287ms/step - accuracy: 0.4750 - loss: 0.7253 - val_accuracy: 0.5333 - val_loss: 0.6937
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 275ms/step - accuracy: 0.5286 - loss: 0.7086 - val_accuracy: 0.5333 - val_loss: 0.6937
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 224ms/step - accuracy: 0.5000 - loss: 0.7311 - val_accuracy: 0.5333 - val_loss: 0.6939
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 215ms/step - accuracy: 0.4857 - loss: 0.7216 - val_accuracy: 0.5167 - val_loss: 0.6939
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 147ms/step - accuracy: 0.5321 - loss: 0.7126 - val_accuracy: 0.5167 - val_loss: 0.6939
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.4821 - loss: 0.7293 - val_accuracy: 0.5500 - val_

In [128]:
test_loss11, test_accuracy11 = model11.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy11)
print("Test Loss:", test_loss11)

Test Accuracy: 0.5333333611488342
Test Loss: 0.693164050579071


In [129]:
model12 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model12.summary()

Model: "sequential_35"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_71 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_72 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_73 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_80 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_87 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_81 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_88 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_82 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_89 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,071,553 (7.90 MB)

 Trainable params: 2,070,529 (7.90 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [130]:
model12.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model12.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.5393 - loss: 0.7978 - val_accuracy: 0.4833 - val_loss: 0.7027
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 278ms/step - accuracy: 0.5750 - loss: 0.7972 - val_accuracy: 0.4833 - val_loss: 0.7033
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 230ms/step - accuracy: 0.5000 - loss: 0.8344 - val_accuracy: 0.4500 - val_loss: 0.7036
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 214ms/step - accuracy: 0.4607 - loss: 0.8636 - val_accuracy: 0.4500 - val_loss: 0.7039
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - accuracy: 0.6000 - loss: 0.7268 - val_accuracy: 0.4500 - val_loss: 0.7044
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 195ms/step - accuracy: 0.5464 - loss: 0.7600 - val_accuracy: 0.4500 - val_loss: 0.7047
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - accuracy: 0.5000 - loss: 0.8100 - val_accuracy: 0.4667 - val_loss: 0.7047
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - accuracy: 0.5393 - loss: 0.7614 - val_accuracy: 0.5000 - val

In [131]:
test_loss12, test_accuracy12 = model12.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy12)
print("Test Loss:", test_loss12)

Test Accuracy: 0.4833333194255829
Test Loss: 0.7027408480644226


In [132]:
model13 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=False),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model13.summary()

Model: "sequential_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_74 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_75 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_90 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_83 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_91 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_84 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_92 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,676,801 (6.40 MB)

 Trainable params: 1,675,777 (6.39 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [133]:
model13.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model13.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 59s 552ms/step - accuracy: 0.4679 - loss: 0.9384 - val_accuracy: 0.5000 - val_loss: 0.7131
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 180ms/step - accuracy: 0.5107 - loss: 0.9079 - val_accuracy: 0.5000 - val_loss: 0.7113
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 191ms/step - accuracy: 0.4893 - loss: 0.9584 - val_accuracy: 0.5167 - val_loss: 0.7094
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step - accuracy: 0.4893 - loss: 0.9589 - val_accuracy: 0.5167 - val_loss: 0.7077
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.5000 - loss: 0.9334 - val_accuracy: 0.5167 - val_loss: 0.7062
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 164ms/step - accuracy: 0.5286 - loss: 0.8871 - val_accuracy: 0.5167 - val_loss: 0.7044
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 221ms/step - accuracy: 0.5286 - loss: 0.8445 - val_accuracy: 0.5000 - val_loss: 0.7026
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 179ms/step - accuracy: 0.5286 - loss: 0.8811 - val_accuracy: 0.5167 - 

In [134]:
test_loss13, test_accuracy13 = model13.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy13)
print("Test Loss:", test_loss13)

Test Accuracy: 0.5666666626930237
Test Loss: 0.6807321906089783


In [140]:
model14 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model14.summary()

Model: "sequential_38"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_78 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_79 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_87 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_95 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,577,217 (6.02 MB)

 Trainable params: 1,576,705 (6.01 MB)

 Non-trainable params: 512 (2.00 KB)

In [141]:
model14.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model14.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 35s 813ms/step - accuracy: 0.4500 - loss: 0.8619 - val_accuracy: 0.5167 - val_loss: 0.7354
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 256ms/step - accuracy: 0.5321 - loss: 0.7805 - val_accuracy: 0.5167 - val_loss: 0.7229
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 188ms/step - accuracy: 0.5250 - loss: 0.7862 - val_accuracy: 0.5333 - val_loss: 0.7117
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - accuracy: 0.5036 - loss: 0.7945 - val_accuracy: 0.5167 - val_loss: 0.7032
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 182ms/step - accuracy: 0.5536 - loss: 0.7491 - val_accuracy: 0.5167 - val_loss: 0.6973
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 174ms/step - accuracy: 0.5929 - loss: 0.7236 - val_accuracy: 0.5500 - val_loss: 0.6917
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.5964 - loss: 0.7176 - val_accuracy: 0.5333 - val_loss: 0.6871
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 174ms/step - accuracy: 0.5821 - loss: 0.7073 - val_accuracy: 0.5833 - 

In [142]:
test_loss14, test_accuracy14 = model14.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy14)
print("Test Loss:", test_loss14)

Test Accuracy: 0.6333333253860474
Test Loss: 0.6698211431503296


In [143]:
model15 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model15.summary()

Model: "sequential_39"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_80 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_81 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_88 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_96 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_89 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_97 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_90 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_98 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_91 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_99 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,872,897 (7.14 MB)

 Trainable params: 1,872,385 (7.14 MB)

 Non-trainable params: 512 (2.00 KB)

In [144]:
model15.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model15.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 45s 645ms/step - accuracy: 0.5357 - loss: 0.7812 - val_accuracy: 0.4500 - val_loss: 0.7054
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 226ms/step - accuracy: 0.4929 - loss: 0.8037 - val_accuracy: 0.4333 - val_loss: 0.7052
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.5000 - loss: 0.8192 - val_accuracy: 0.4167 - val_loss: 0.7053
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.5000 - loss: 0.7963 - val_accuracy: 0.4000 - val_loss: 0.7052
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.5250 - loss: 0.7939 - val_accuracy: 0.3667 - val_loss: 0.7050
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 233ms/step - accuracy: 0.4393 - loss: 0.8423 - val_accuracy: 0.3667 - val_loss: 0.7047
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 216ms/step - accuracy: 0.4750 - loss: 0.8231 - val_accuracy: 0.3833 - val_loss: 0.7045
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 219ms/step - accuracy: 0.5179 - loss: 0.8036 - val_accuracy: 0.4000 - va

In [145]:
test_loss15, test_accuracy15 = model15.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy15)
print("Test Loss:", test_loss15)

Test Accuracy: 0.44999998807907104
Test Loss: 0.7042970657348633


In [146]:
model16 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model16.summary()

Model: "sequential_40"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_82 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_83 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_84 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_92 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_100 (Dense)                    │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,970,945 (7.52 MB)

 Trainable params: 1,970,945 (7.52 MB)

 Non-trainable params: 0 (0.00 B)

In [147]:
model16.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model16.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.5107 - loss: 0.7010 - val_accuracy: 0.5000 - val_loss: 0.7025
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 227ms/step - accuracy: 0.5679 - loss: 0.6823 - val_accuracy: 0.4667 - val_loss: 0.7003
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 213ms/step - accuracy: 0.5714 - loss: 0.6714 - val_accuracy: 0.5500 - val_loss: 0.6987
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 233ms/step - accuracy: 0.5464 - loss: 0.6902 - val_accuracy: 0.5167 - val_loss: 0.6975
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step - accuracy: 0.5893 - loss: 0.6681 - val_accuracy: 0.5167 - val_loss: 0.6963
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.5571 - loss: 0.6752 - val_accuracy: 0.5167 - val_loss: 0.6951
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 156ms/step - accuracy: 0.5607 - loss: 0.6788 - val_accuracy: 0.5167 - val_loss: 0.6937
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 195ms/step - accuracy: 0.5857 - loss: 0.6709 - val_accuracy: 0.5167 - val

In [148]:
test_loss16, test_accuracy16 = model16.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy16)
print("Test Loss:", test_loss16)

Test Accuracy: 0.6666666865348816
Test Loss: 0.6675261855125427


In [149]:
model17 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model17.summary()

Model: "sequential_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_85 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_86 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_87 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_93 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_101 (Dense)                    │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,972,993 (7.53 MB)

 Trainable params: 1,971,969 (7.52 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [150]:
model17.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model17.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.4821 - loss: 0.8349 - val_accuracy: 0.5167 - val_loss: 0.6962
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 327ms/step - accuracy: 0.5036 - loss: 0.7833 - val_accuracy: 0.5000 - val_loss: 0.6949
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 243ms/step - accuracy: 0.5036 - loss: 0.7752 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 241ms/step - accuracy: 0.5321 - loss: 0.7653 - val_accuracy: 0.5167 - val_loss: 0.6911
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 242ms/step - accuracy: 0.5607 - loss: 0.7198 - val_accuracy: 0.5167 - val_loss: 0.6894
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 269ms/step - accuracy: 0.5607 - loss: 0.7017 - val_accuracy: 0.4833 - val_loss: 0.6875
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 253ms/step - accuracy: 0.5357 - loss: 0.6952 - val_accuracy: 0.5000 - val_loss: 0.6855
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step - accuracy: 0.6000 - loss: 0.6893 - val_accuracy: 0.5000 - val

In [151]:
test_loss17, test_accuracy17 = model17.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy17)
print("Test Loss:", test_loss17)

Test Accuracy: 0.550000011920929
Test Loss: 0.6757776737213135


In [152]:
model18 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model18.summary()

Model: "sequential_42"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_88 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_22               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_89 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_23               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_90 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_94 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_102 (Dense)                    │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_95 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_103 (Dense)                    │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,005,761 (7.65 MB)

 Trainable params: 2,004,737 (7.65 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [153]:
model18.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model18.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - accuracy: 0.4964 - loss: 0.8820 - val_accuracy: 0.4500 - val_loss: 0.7136
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 168ms/step - accuracy: 0.4750 - loss: 0.8752 - val_accuracy: 0.4333 - val_loss: 0.7126
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 230ms/step - accuracy: 0.5357 - loss: 0.8122 - val_accuracy: 0.4500 - val_loss: 0.7119
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.5643 - loss: 0.7465 - val_accuracy: 0.4167 - val_loss: 0.7116
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 197ms/step - accuracy: 0.5750 - loss: 0.7803 - val_accuracy: 0.4333 - val_loss: 0.7113
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 243ms/step - accuracy: 0.5000 - loss: 0.8558 - val_accuracy: 0.4500 - val_loss: 0.7108
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 213ms/step - accuracy: 0.5286 - loss: 0.7902 - val_accuracy: 0.4500 - val_loss: 0.7101
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 236ms/step - accuracy: 0.5750 - loss: 0.7286 - val_accuracy: 0.4333 - val_

In [154]:
test_loss18, test_accuracy18 = model18.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy18)
print("Test Loss:", test_loss18)

Test Accuracy: 0.44999998807907104
Test Loss: 0.7101463675498962


In [157]:
model19 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model19.summary()

Model: "sequential_44"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_94 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_26               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_95 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_27               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_96 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_100 (Dropout)                │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_108 (Dense)                    │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_101 (Dropout)                │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_109 (Dense)                    │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_102 (Dropout)                │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_110 (Dense)                    │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_103 (Dropout)                │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_111 (Dense)                    │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,268,673 (8.65 MB)

 Trainable params: 2,267,649 (8.65 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [158]:
model19.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model19.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 47s 705ms/step - accuracy: 0.5036 - loss: 0.8406 - val_accuracy: 0.4833 - val_loss: 0.6975
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - accuracy: 0.4714 - loss: 0.8808 - val_accuracy: 0.4167 - val_loss: 0.6975
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step - accuracy: 0.4786 - loss: 0.8623 - val_accuracy: 0.4500 - val_loss: 0.6976
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.5214 - loss: 0.8280 - val_accuracy: 0.4000 - val_loss: 0.6976
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.5107 - loss: 0.8437 - val_accuracy: 0.3833 - val_loss: 0.6973
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - accuracy: 0.4607 - loss: 0.8517 - val_accuracy: 0.4000 - val_loss: 0.6971
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.5000 - loss: 0.8611 - val_accuracy: 0.4167 - val_loss: 0.6970
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 190ms/step - accuracy: 0.5571 - loss: 0.7449 - val_accuracy: 0.4167 - v

In [159]:
test_loss19, test_accuracy19 = model19.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy19)
print("Test Loss:", test_loss19)

Test Accuracy: 0.46666666865348816
Test Loss: 0.6965444087982178


In [160]:
model20 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=True),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model20.summary()

Model: "sequential_45"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_97 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_98 (GRU)                         │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_99 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_104 (Dropout)                │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_112 (Dense)                    │ (None, 1024)                │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_105 (Dropout)                │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_113 (Dense)                    │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_106 (Dropout)                │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_114 (Dense)                    │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_107 (Dropout)                │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_115 (Dense)                    │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_108 (Dropout)                │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_116 (Dense)                    │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,923,009 (11.15 MB)

 Trainable params: 2,923,009 (11.15 MB)

 Non-trainable params: 0 (0.00 B)

In [161]:
model20.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model20.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.4929 - loss: 0.7141 - val_accuracy: 0.4667 - val_loss: 0.6964
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 247ms/step - accuracy: 0.4571 - loss: 0.7395 - val_accuracy: 0.4333 - val_loss: 0.6961
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 225ms/step - accuracy: 0.4607 - loss: 0.7353 - val_accuracy: 0.4167 - val_loss: 0.6959
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 256ms/step - accuracy: 0.4571 - loss: 0.7201 - val_accuracy: 0.4333 - val_loss: 0.6958
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 232ms/step - accuracy: 0.4857 - loss: 0.7053 - val_accuracy: 0.4167 - val_loss: 0.6957
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 142ms/step - accuracy: 0.5321 - loss: 0.7011 - val_accuracy: 0.4333 - val_loss: 0.6959
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 188ms/step - accuracy: 0.5429 - loss: 0.6907 - val_accuracy: 0.4500 - val_loss: 0.6960
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 0.4929 - loss: 0.7147 - val_accuracy: 0.4667 - val

In [162]:
test_loss20, test_accuracy20 = model20.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy20)
print("Test Loss:", test_loss20)

Test Accuracy: 0.4166666567325592
Test Loss: 0.6957305073738098


In [163]:
model21 = Sequential([
    Input(shape=(num_frames, num_features)),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=True),
    BatchNormalization(),
    GRU(256, return_sequences=False),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid'),
])
model21.summary()

Model: "sequential_46"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_100 (GRU)                        │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_28               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_101 (GRU)                        │ (None, 16, 256)             │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_29               │ (None, 16, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_102 (GRU)                        │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_109 (Dropout)                │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_117 (Dense)                    │ (None, 1024)                │         263,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_110 (Dropout)                │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_118 (Dense)                    │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_111 (Dropout)                │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_119 (Dense)                    │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_112 (Dropout)                │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_120 (Dense)                    │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_113 (Dropout)                │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_121 (Dense)                    │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,925,057 (11.16 MB)

 Trainable params: 2,924,033 (11.15 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [164]:
model21.compile(
    optimizer=Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True, verbose=1)
model21.fit(train_embeddings, y_train, 
           validation_data=(val_embeddings, y_val),
           epochs=500, batch_size=batch_size,
           callbacks=[estop], verbose=1)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 37s 852ms/step - accuracy: 0.4750 - loss: 0.8477 - val_accuracy: 0.5000 - val_loss: 0.6984
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 234ms/step - accuracy: 0.4964 - loss: 0.8227 - val_accuracy: 0.5000 - val_loss: 0.6981
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 223ms/step - accuracy: 0.4679 - loss: 0.8512 - val_accuracy: 0.5000 - val_loss: 0.6981
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 245ms/step - accuracy: 0.4929 - loss: 0.8138 - val_accuracy: 0.5000 - val_loss: 0.6982
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 233ms/step - accuracy: 0.5214 - loss: 0.8113 - val_accuracy: 0.5000 - val_loss: 0.6982
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - accuracy: 0.4750 - loss: 0.8256 - val_accuracy: 0.5000 - val_loss: 0.6981
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.4857 - loss: 0.7945 - val_accuracy: 0.5000 - val_loss: 0.6980
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.5179 - loss: 0.8211 - val_accuracy: 0.4833 - v

In [165]:
test_loss21, test_accuracy21 = model21.evaluate(test_embeddings, y_test, verbose=0)
print("Test Accuracy:", test_accuracy21)
print("Test Loss:", test_loss21)

Test Accuracy: 0.46666666865348816
Test Loss: 0.6977048516273499


In [166]:
model_records = [
    (model1, test_accuracy1, test_loss1),
    (model2, test_accuracy2, test_loss2),
    (model3, test_accuracy3, test_loss3),
    (model4, test_accuracy4, test_loss4),
    (model5, test_accuracy5, test_loss5),
    (model6, test_accuracy6, test_loss6),
    (model7, test_accuracy7, test_loss7),
    (model8, test_accuracy8, test_loss8),
    (model9, test_accuracy9, test_loss9),
    (model10, test_accuracy10, test_loss10),
    (model11, test_accuracy11, test_loss11),
    (model12, test_accuracy12, test_loss12),
    (model13, test_accuracy13, test_loss13),
    (model14, test_accuracy14, test_loss14),
    (model15, test_accuracy15, test_loss15),
    (model16, test_accuracy16, test_loss16),
    (model17, test_accuracy17, test_loss17),
    (model18, test_accuracy18, test_loss18),
    (model19, test_accuracy19, test_loss19),
    (model20, test_accuracy20, test_loss20),
    (model21, test_accuracy21, test_loss21),
]
best_acc = 0
best_loss = 0
best_model = None
for model,acc,loss in model_records:
    if acc>best_acc:
        best_acc = acc
        best_loss = loss
        best_model = model
    elif acc==best_acc:
        if loss<best_loss:
            best_loss=loss
            best_model=model
print("Best accuracy:", best_acc)
print("Best loss:", best_loss)
best_model.summary()

Best accuracy: 0.6666666865348816
Best loss: 0.6623032689094543


Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_51 (GRU)                         │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_52 (GRU)                         │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_58 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_63 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,728,581 (18.04 MB)

 Trainable params: 1,576,193 (6.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,152,388 (12.03 MB)

In [167]:
best_model.save('artifacts/best_mobilenetv2_embeddings_gru.keras')

* Adding more FC layers and more GRUs seem to improve the performance, but there's no definite pattern.
  - In case of single GRU architectures adding one extra Dense layer (other than the Output layer) gave the best performance.
  - In cases of architectures with either double- and triple-stacked GRUs, having no Dense layers (except of Output layer) gave the best performance. These were also the models that gave the highest seen accuracies amongst all models that were tried.
* For consistency dropouts are added between GRU & FC and between FCs in all the architectures.
* Every architecture is experimented with and without BatchNormalization in between the GRUs. Only in 3 out of 8 cases did BatchNormalization bring some improvement to the performance. So, it's not particularly reliable.
* The best performing achitecture is (`model3`) one with 2 GRUs and an output FC layer (with dropout between GRU-FC), with an accuracy of **66.7%** and loss of **0.6623**. The second best performing model (`model16`), which also has **66.7%** accuracy but a slightly higher loss of **0.6675**, consists of 3 GRUs and an output FC layer (with dropout between GRU-FC).
* Other well performing models are-
  1. `model14`: 2 GRUs with BatchNormalization between the pair & output FC layer (with dropout between GRU-FC) -> **63.3%** accuracy
  2. `model2`: 1 GRU, 1 FC, & output FC layer (with dropouts between GRU-FC & FC-FC) -> **61.7%** accuracy
  3. `model9`: 3 GRUs, 1 FC, & output FC layer (with dropouts between GRU-FC & FC-FC) -> **60%** accuracy and **0.6747** loss
  4. `model10`: 3 GRUs, 2 FC, & output FC layer (with dropouts between GRU-FC & FC-FC) -> **60%** accuracy and **0.6774** loss

**It should also be noted that these models were extremely quick to train, which made trying out several different architectures very easy.**

* This architecture can be tuned further with the inclusion of l1/l2 regularization, and different dropout rates, optimizers, and scheduled learning rates.
  - Since, the training is already quite fast, adding momentum may not necessarily help.
  - Scheduling the learning rates and making it slower after a while may have higher scope of giving an improvement, even though all of the above trials used a small learning rate of 1e-5 (although not included in this notebook, bigger learning rates - 0.01, 0.001, 0.0001 - were also experimented with but discarded because the accuracies were lower than seen with 1e-5, hence, perhaps even smaller learning rates could help for this usecase).

* Furthermore, we could also try other pretrained models for obtaining the embeddings. But we need to take care of the sizes of the images that are fed into those models.
  - densenet, efficientnetb0, mobilenetv2, resnet50 -> 224x224
  - xception, inceptionv3 -> 299x299
  - efficientnetb3 -> 300x300

**Regardless of the model and technique used, we don't appear to get any exceptionally high values of accuracy (like percentages in 80s and 90s). This is probably due to the small size of the dataset and maybe also because of the nature of the data. These deepfake videos aren't entirely AI-generated, instead the faces/expressions alone, of the people in the videos, have been swapped/altered. So, our model needs to identify the fakeness of the video from a very small spatial range of the frames. That is a sensitive task, and a model will only be able to handle that if it were fed a significantly large dataset to learn from.**

In [14]:
classifier = keras.models.load_model('artifacts/best_mobilenetv2_embeddings_gru.keras')

In [17]:
X_test0_frames, _ = util.convert_3d_to_2d(split=1, data=(np.array([X_test[0]]), _))
X_test[0].shape, X_test0_frames.shape

((16, 224, 224, 3), (16, 224, 224, 3))

In [18]:
X_test0_embeddings = get_embeddings(X_test0_frames, 1, 1, num_frames)
X_test0_embeddings.shape

16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step
Embeddings shape: (16, 1280)


(1, 16, 1280)

In [19]:
pred = classifier.predict(X_test0_embeddings)
pred.shape, pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


((1, 1), array([[0.7866799]], dtype=float32))

In [20]:
pred = (pred>=0.5).astype(int)
pred

array([[1]])

In [21]:
y_test[0]

0

In [23]:
pred[0][0]

1